In [2]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
import csv
#from sklearn.model_selection import train_test_split
#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler
#from sklearn.linear_model import LogisticRegression, RidgeClassifier
#from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
import pickle 


In [3]:

# Initialize MediaPipe Holistic and Drawing utilities
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh  


In [5]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [7]:
data=pd.read_csv("cord.csv",nrows=1)


In [13]:
capture = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while capture.isOpened():
        ret, frame = capture.read()
        if not ret:
            print("Failed to capture frame.")
            break

        # Flip the frame horizontally
        flip_frame = cv2.flip(frame, 1)

        # Convert the frame to RGB for processing
        image = cv2.cvtColor(flip_frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False  # Optimize by marking the image as read-only

        # Process the frame with MediaPipe Holistic
        results = holistic.process(image)

        # Convert the frame back to BGR for OpenCV display
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw face landmarks if detected
        if results.face_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.face_landmarks,
                mp_face_mesh.FACEMESH_CONTOURS,  
                mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1),
            )

        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.right_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
            
            )

        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.left_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,  
            )
        

        # Draw pose landmarks if detected
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_holistic.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=3),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
            )
            
        try:
            # Ensure pose landmarks exist
            if results.pose_landmarks:
                pose = results.pose_landmarks.landmark
                pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            else:
                pose_row = []

            # Ensure face landmarks exist
            if results.face_landmarks:
                face = results.face_landmarks.landmark
                face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            else:
                face_row = []
            
            # Combine pose and face data
            row = pose_row + face_row

            if row:  # Check if row contains data
                # Ensure X is a DataFrame with correct columns
                X = pd.DataFrame([row], columns=data.columns[1:])
                
                # Make prediction and probability
                language_class = model.predict(X)[0]
                language_prob = model.predict_proba(X)[0].max()  # Get the maximum probability

                # Display "No Gesture" if probability is less than 0.5
                if language_prob < 0.70:
                    language_class = "No Gesture"
                    language_prob = 0.0  # Set probability to 0 if no gesture is detected

                # Top-right corner positioning
                top_right_coords = (image.shape[1] - 250, 30)  # 250 pixels width from right and 30 pixels down from top

                # Draw the background box for prediction
                cv2.rectangle(image, 
                              (top_right_coords[0], top_right_coords[1] - 20), 
                              (top_right_coords[0] + 250, top_right_coords[1] + 30), 
                              (245, 117, 16), -1)

                # Display the predicted class at the top-right corner
                cv2.putText(image, 'CLASS', (top_right_coords[0] + 10, top_right_coords[1] + 15), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, language_class, 
                            (top_right_coords[0] + 10, top_right_coords[1] + 40), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        except Exception as e:
            print(f"Error: {e}")

        # Display the annotated frame (always executed)
        cv2.imshow("Live Feed", image)

        # Break the loop when 'q' is pressed (always executed)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release resources
capture.release()
cv2.destroyAllWindows()
